In [30]:
%%writefile app.py
import streamlit as st
import pandas as pd
from io import BytesIO
import re
import time
from openpyxl import load_workbook

st.title("Excel Form Processor with Categories")

# --- Initialize session state ---
if 'selected_sheet' not in st.session_state:
    st.session_state.selected_sheet = "Please select"
if 'column_mapping' not in st.session_state:
    st.session_state.column_mapping = {}

# Step 1: Upload Excel file
uploaded_file = st.file_uploader("Upload an Excel file", type=["xlsx"])

if uploaded_file:
    # Read Excel sheets
    xls = pd.ExcelFile(uploaded_file)
    sheet_names = xls.sheet_names

    st.write("### Select the sheet containing form fields:")

    # Sheet dropdown
    selected_sheet = st.selectbox(
        "Select sheet",
        options=["Please select"] + sheet_names,
        index=0 if st.session_state.selected_sheet == "Please select" else sheet_names.index(st.session_state.selected_sheet)+1
    )

    if selected_sheet != "Please select":
        st.session_state.selected_sheet = selected_sheet
        st.success(f"Selected sheet: {selected_sheet}")
        df = pd.read_excel(uploaded_file, sheet_name=selected_sheet)
        st.write("### Select columns for required fields:")

        required_fields = ["Field Name", "Field Order", "Field Type", "Form Name"]

        # Column selection dropdowns
        for field in required_fields:
            if field not in st.session_state.column_mapping:
                st.session_state.column_mapping[field] = "Please select"

            selected_col = st.selectbox(
                f"Select column for {field}",
                options=["Please select"] + list(df.columns),
                index=0 if st.session_state.column_mapping[field] == "Please select"
                      else list(df.columns).index(st.session_state.column_mapping[field])+1,
                key=f"col_select_{field}"
            )

            st.session_state.column_mapping[field] = selected_col

        column_mapping = st.session_state.column_mapping

        if all(v != "Please select" for v in column_mapping.values()):
            st.success("All required columns selected!")

            form_col = column_mapping["Form Name"]
            field_name_col = column_mapping["Field Name"]
            field_order_col = column_mapping["Field Order"]
            field_type_col = column_mapping["Field Type"]

            all_forms = df[form_col].dropna().unique()

            # Step: Select which forms to process
            selected_forms = st.multiselect(
                "Select forms to process (or leave all selected for all forms):",
                options=all_forms,
                default=all_forms
            )

            # Step: Select additional category columns
            other_cols = [c for c in df.columns if c not in required_fields]
            selected_category_cols = st.multiselect(
                "Select additional columns (0/1) to use for Categories (optional):",
                options=other_cols
            )

            if st.button("Generate Excel Report"):
                output = BytesIO()
                total_forms = len(selected_forms)
                progress_bar = st.progress(0)

                # Load original workbook to preserve existing sheets
                uploaded_file.seek(0)
                wb = load_workbook(uploaded_file)

                for i, form in enumerate(selected_forms):
                    # Filter by form and sort by Field Order
                    form_df = df[df[form_col] == form].sort_values(by=field_order_col)
                    final_df = form_df[[field_name_col, field_type_col, field_order_col]].copy()

                    # Add categories if user selected any columns
                    if selected_category_cols:
                        categories_list = []
                        for idx, row in form_df.iterrows():
                            selected = [col for col in selected_category_cols if row[col] == 1]
                            categories_list.append(",".join(selected) if selected else "")
                        final_df['Categories'] = categories_list

                    # Sanitize sheet name
                    sheet_name = re.sub(r'[\\/*?:\\[\\]]', '_', str(form))[:31]

                    # Remove existing sheet if present
                    if sheet_name in wb.sheetnames:
                        std = wb[sheet_name]
                        wb.remove(std)

                    # Create new sheet
                    ws = wb.create_sheet(title=sheet_name)

                    # Write headers
                    for c_idx, header in enumerate(final_df.columns, start=1):
                        ws.cell(row=1, column=c_idx, value=header)

                    # Write data
                    for r_idx, row in enumerate(final_df.values, start=2):
                        for c_idx, value in enumerate(row, start=1):
                            ws.cell(row=r_idx, column=c_idx, value=value)

                    # Update progress bar
                    progress = int((i + 1) / total_forms * 100)
                    progress_bar.progress(progress)
                    time.sleep(0.05)

                # Save modified workbook to BytesIO
                wb.save(output)
                st.success("Original Excel modified successfully with Categories!")
                st.download_button(
                    label="Download Modified Original Excel",
                    data=output.getvalue(),
                    file_name="modified_original_with_categories.xlsx",
                    mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
                )


Overwriting app.py


In [31]:
# Install packages (if not installed already)
!pip install streamlit pyngrok openpyxl --quiet

from pyngrok import ngrok
import time

# Kill previous tunnels
ngrok.kill()

# Set your ngrok token
ngrok.set_auth_token("35gXlCpbnwCAIJ9I2xiA1GXDu2u_3UoQf6QeGYhFC9MxYapgo")

# Start Streamlit in the background
get_ipython().system_raw("streamlit run app.py &")

# Wait a few seconds for Streamlit to start
time.sleep(5)

# Open ngrok tunnel
public_url = ngrok.connect(8501)
print("Streamlit app running at:", public_url)


Streamlit app running at: NgrokTunnel: "https://armless-twirly-alanna.ngrok-free.dev" -> "http://localhost:8501"
